In [2]:
%matplotlib inline

In [3]:

import os
import numpy as np
import json
from ipywidgets import interact
import matplotlib.pyplot as plt
import torch
import random
import re
from tqdm.auto import tqdm
from espnet.asr.pytorch_backend.asr import load_trained_model
if 'cd' not in globals():
    cd = True
    os.chdir('..')
import matplotlib
matplotlib.use('TkAgg')

In [4]:
def get_cer(plot_dir):
    data = []
    for folder in os.listdir(plot_dir):
            if folder.startswith('snapshot.ep.'):
                ep = int(folder.strip('snapshot.ep.'))
                with open(f'{plot_dir}/{folder}/result.txt', 'r', encoding="utf-8") as f:
                    for l in f:
                        if 'Sum/Avg' in l:
                            pter = float(l.strip().strip('|').strip().split()[-2])
                            data.append((ep, pter))
                            break

    data = sorted(data, key=lambda x: x[0])
    eps, pters = zip(*data)
    return np.array(eps), np.array(pters)

def get_cer_with_cache(plot_dir, cache_data):
    if plot_dir in cache_data:
        eps, pters = cache_data[plot_dir]
    else:
        eps, pters = get_cer(plot_dir)
        cache_data[plot_dir] = eps, pters
    return eps, pters


In [5]:
cache_data = {}
langs = ['Croatian', 'Polish', 'Spanish', 'Portuguese', 'Turkish', 'German', 'Bulgarian', 'Thai', 'Mandarin', 'French', 'Czech', '203', '101', '505', '404', '402', '307', '206', '107', '103']
lang_labels = ['CR', 'PL', 'SP', 'PO', 'TU', 'GE', 'BG', 'TH', 'CH', 'FR', 'CZ', '203', '101', 'N', '404', '402', '307', '206', '107', '103']
train_langs = ['Portuguese', 'Turkish', 'German', 'Bulgarian', 'Thai', 'Mandarin', 'French', 'Czech', '505', '404', '402', '307', '206', '107', '103']
lang2label = {lang: label for lang, label in zip(langs, lang_labels)}

In [ ]:
max_epoch=30
test_langs = ['Croatian', 'Polish', 'Spanish', '203', '101'] + train_langs
chosen_ep = 29
colors = cm.rainbow(np.linspace(0, 1, len(test_langs)))
# plt.figure(figsize=(20,15))
cross_pters = []
for i, lang in enumerate(test_langs):
    label = lang2label[lang]
    plot_dir = f'exp/train_pytorch_wav2vecfexlembft/plot_mask_eval_{lang}_{label}_decode'
#     base_plot_dir = f'exp/train_pytorch_wav2vecfexlembadv/plot_eval_{lang}_{label}_decode'
#     if not os.path.exists(base_plot_dir): 
#         continue
    eps, pters = get_cer_with_cache(plot_dir, cache_data)
#     plt.plot(eps[:max_epoch], pters[:max_epoch], label=f'{lang}_'+label, color=colors[i])
    min_ep = eps[np.argmin(pters)]
    min_pter = min(pters)
#     print(f'lgcn Lang: {lang} min_ep: {min_ep} min_pter {min_pter}')
    print(f'lemb  Lang: {lang} ep: {min_ep} min_pter {min_pter}')

#     print(f'base Lang: {lang} min_ep: {min_ep} min_pter {min_pter}')
print(','.join(test_langs))
print(','.join(cross_pters))
# plt.legend()
# plt.title(f'LGCN MASK PTER {eps[chosen_ep]}')
# plt.show()



In [21]:
i = 14
model, train_args = load_trained_model(f'exp/train_pytorch_wav2vecfexlembft/results/snapshot.ep.{i}')
device = torch.device('cuda:1')
model = model.float()
model1 = model.to(device)
for n, p in model1.feature_extractor.named_parameters():
    print(n, p)
    p1 = p
    break

conv_layers.0.0.weight Parameter containing:
tensor([[[-9.9304e-02, -3.9093e-02,  1.5503e-01,  ...,  2.1631e-01,
           2.3183e-02,  2.7759e-01]],

        [[ 9.3201e-02, -8.4900e-02, -2.8931e-01,  ..., -1.4417e-01,
           1.9194e-02,  2.4133e-01]],

        [[ 1.2659e-01, -2.5732e-01,  2.1460e-01,  ..., -2.2522e-01,
           2.9663e-01, -1.2927e-01]],

        ...,

        [[-2.1680e-01, -6.2347e-02,  1.5552e-01,  ..., -2.3279e-01,
           1.7435e-05,  4.0747e-01]],

        [[-1.8237e-01, -1.4844e-01,  7.2083e-02,  ...,  8.0994e-02,
           2.2376e-02,  2.8076e-01]],

        [[ 2.4622e-01,  1.3763e-02, -1.2627e-02,  ..., -9.3427e-03,
           1.4844e-01,  1.7493e-01]]], device='cuda:1', requires_grad=True)


In [13]:
i = 30
model, train_args = load_trained_model(f'exp/train_pytorch_wav2vecfexlembglottoonly/results/snapshot.ep.{i}')
device = torch.device('cuda:1')
model = model.float()
model = model.to(device)
for n, p in model.feature_extractor.named_parameters():
    print(n, p)
    p2 = p
    break

conv_layers.0.0.weight Parameter containing:
tensor([[[-9.9304e-02, -3.9093e-02,  1.5503e-01,  ...,  2.1631e-01,
           2.3178e-02,  2.7759e-01]],

        [[ 9.3201e-02, -8.4900e-02, -2.8931e-01,  ..., -1.4417e-01,
           1.9196e-02,  2.4133e-01]],

        [[ 1.2659e-01, -2.5732e-01,  2.1460e-01,  ..., -2.2522e-01,
           2.9663e-01, -1.2927e-01]],

        ...,

        [[-2.1680e-01, -6.2347e-02,  1.5552e-01,  ..., -2.3279e-01,
           2.6882e-05,  4.0747e-01]],

        [[-1.8237e-01, -1.4844e-01,  7.2083e-02,  ...,  8.0994e-02,
           2.2385e-02,  2.8076e-01]],

        [[ 2.4622e-01,  1.3771e-02, -1.2619e-02,  ..., -9.3384e-03,
           1.4844e-01,  1.7493e-01]]], device='cuda:1')


In [22]:
i = 14
model, train_args = load_trained_model(f'exp/train_pytorch_wav2vecfexlembglottoonly/results/snapshot.ep.{i}')
device = torch.device('cuda:1')
model = model.float()
model3 = model.to(device)

for n, p in model3.feature_extractor.named_parameters():
    print(n, p)
    p3 = p
    break

conv_layers.0.0.weight Parameter containing:
tensor([[[-9.9304e-02, -3.9093e-02,  1.5503e-01,  ...,  2.1631e-01,
           2.3178e-02,  2.7759e-01]],

        [[ 9.3201e-02, -8.4900e-02, -2.8931e-01,  ..., -1.4417e-01,
           1.9196e-02,  2.4133e-01]],

        [[ 1.2659e-01, -2.5732e-01,  2.1460e-01,  ..., -2.2522e-01,
           2.9663e-01, -1.2927e-01]],

        ...,

        [[-2.1680e-01, -6.2347e-02,  1.5552e-01,  ..., -2.3279e-01,
           2.6882e-05,  4.0747e-01]],

        [[-1.8237e-01, -1.4844e-01,  7.2083e-02,  ...,  8.0994e-02,
           2.2385e-02,  2.8076e-01]],

        [[ 2.4622e-01,  1.3771e-02, -1.2619e-02,  ..., -9.3384e-03,
           1.4844e-01,  1.7493e-01]]], device='cuda:1')


In [23]:
torch.all(p1 == p2)

tensor(False, device='cuda:1')

In [25]:
p1

Parameter containing:
tensor([[[-9.9304e-02, -3.9093e-02,  1.5503e-01,  ...,  2.1631e-01,
           2.3183e-02,  2.7759e-01]],

        [[ 9.3201e-02, -8.4900e-02, -2.8931e-01,  ..., -1.4417e-01,
           1.9194e-02,  2.4133e-01]],

        [[ 1.2659e-01, -2.5732e-01,  2.1460e-01,  ..., -2.2522e-01,
           2.9663e-01, -1.2927e-01]],

        ...,

        [[-2.1680e-01, -6.2347e-02,  1.5552e-01,  ..., -2.3279e-01,
           1.7435e-05,  4.0747e-01]],

        [[-1.8237e-01, -1.4844e-01,  7.2083e-02,  ...,  8.0994e-02,
           2.2376e-02,  2.8076e-01]],

        [[ 2.4622e-01,  1.3763e-02, -1.2627e-02,  ..., -9.3427e-03,
           1.4844e-01,  1.7493e-01]]], device='cuda:1', requires_grad=True)

In [28]:
p3

Parameter containing:
tensor([[[-9.9304e-02, -3.9093e-02,  1.5503e-01,  ...,  2.1631e-01,
           2.3178e-02,  2.7759e-01]],

        [[ 9.3201e-02, -8.4900e-02, -2.8931e-01,  ..., -1.4417e-01,
           1.9196e-02,  2.4133e-01]],

        [[ 1.2659e-01, -2.5732e-01,  2.1460e-01,  ..., -2.2522e-01,
           2.9663e-01, -1.2927e-01]],

        ...,

        [[-2.1680e-01, -6.2347e-02,  1.5552e-01,  ..., -2.3279e-01,
           2.6882e-05,  4.0747e-01]],

        [[-1.8237e-01, -1.4844e-01,  7.2083e-02,  ...,  8.0994e-02,
           2.2385e-02,  2.8076e-01]],

        [[ 2.4622e-01,  1.3771e-02, -1.2619e-02,  ..., -9.3384e-03,
           1.4844e-01,  1.7493e-01]]], device='cuda:1')

In [20]:
for (n1, p1), (n3, p3) in zip(model1.feature_extractor.named_parameters(), model3.feature_extractor.named_parameters()):
    print(torch.all(p1==p3))

tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
tensor(False, device='cuda:1')
